# CSCI 8523 Fall 2022 - predicting global crop yields from MODIS
Tim Bodin, Maitrayee Deka, Varun Sethi, Josh Spitzer-Resnick

## imports


In [ ]:
from google.colab import drive # drive.mount
import pickle
import numpy as np
import pandas as pd

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load modis_per_year.pkl (5, 10, 20 bins, avg, yields, years)
path = '/content/drive/Shareddrives/CSCI 8523 F22/Data/'

with open(path + 'modis_per_year.pkl', 'rb') as inp:
    modis_per_year = pickle.load(inp)

## make dataframe

only using days 49-257 for each year

In [ ]:
modis_per_year.keys()

dict_keys([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [ ]:
modis_per_year[2000].keys()

dict_keys(['yield_corn', 'yield_soybean', 'yield_wheat', 'avg', 'bins_5', 'bins_10', 'bins_20', 'bins_5_numpy', 'bins_10_numpy', 'bins_20_numpy'])

In [ ]:
modis_per_year[2000]['yield_corn']

8.591008323

### get into lists

In [ ]:
modis_per_year.keys() # years
yield_corn = []
yield_soybean = []
yield_wheat = []

for year in range(2000, 2023):
  yield_corn.append( modis_per_year[year]['yield_corn'] )
  yield_soybean.append( modis_per_year[year]['yield_soybean'] )
  yield_wheat.append( modis_per_year[year]['yield_wheat'] )

In [ ]:
# unused

day_49_avg = [] # index 3
day_65_avg = []
day_81_avg = []
day_97_avg = []
day_113_avg = []
day_129_avg = []
day_145_avg = []
day_161_avg = []
day_177_avg = []
day_193_avg = []
day_209_avg = []
day_225_avg = []
day_241_avg = []
day_257_avg = [] # index 17

for day in range(1,369,16):
  index = int( (day-1)/16 )
  print(day, index)
  
  for year in range(2000, 2023):
    # print('\t', year)
    if (year == 2000 and day < 49) or (year == 2022 and day > 257):
      print('\t', year)
      continue
    if year == 2000:
      print('\t', year, modis_per_year[year]['avg'][index-3] )
    else:
      print('\t', year, modis_per_year[year]['avg'][index] )

1 0
	 2000
	 2001 0.19162424066942785
	 2002 0.26772451794600916
	 2003 0.2694568539915957
	 2004 0.2503762461244928
	 2005 0.19419622218497276
	 2006 0.2882560742364661
	 2007 0.2836591336223753
	 2008 0.22579137774288474
	 2009 0.22047739550810794
	 2010 0.1991291507195959
	 2011 0.24415001271671238
	 2012 0.29007974206318266
	 2013 0.21635022765449557
	 2014 0.20278640269270226
	 2015 0.17994227631312948
	 2016 0.2614042655997219
	 2017 0.25484388743686975
	 2018 0.1665254396802315
	 2019 0.26565996291541866
	 2020 0.2686340957770858
	 2021 0.21221305289034792
	 2022 0.23520735943870383
17 1
	 2000
	 2001 0.22125813093224023
	 2002 0.24960485762418064
	 2003 0.18760535699324835
	 2004 0.2297875706841101
	 2005 0.21705249726355066
	 2006 0.272741190375843
	 2007 0.18075401805962205
	 2008 0.20571116323079502
	 2009 0.20251819756507952
	 2010 0.21816944900174176
	 2011 0.19336666482025217
	 2012 0.22223011541808252
	 2013 0.26307402255911794
	 2014 0.19223453141845095
	 2015 0.2144016

In [ ]:
len( modis_per_year[2000]['avg'][:14] ) # end early, index starts late # 20 total
len( modis_per_year[2001]['avg'][3:17] ) # start late, end early # 23 total
len( modis_per_year[2022]['avg'][3:] ) # start late, index ends early # 17 total

14

In [ ]:
list(range(49, 273, 16))

[49, 65, 81, 97, 113, 129, 145, 161, 177, 193, 209, 225, 241, 257]

In [ ]:
def get_bin_column_list(bins):
  bin_days = []
  for day in list(range(49, 273, 16)):
    for bin in range(bins):
      bin_days.append( '{}_{}'.format(day, bin) )
  return bin_days

In [ ]:
print( get_bin_column_list(5), '\n', get_bin_column_list(10), '\n', get_bin_column_list(20) )

['49_0', '49_1', '49_2', '49_3', '49_4', '65_0', '65_1', '65_2', '65_3', '65_4', '81_0', '81_1', '81_2', '81_3', '81_4', '97_0', '97_1', '97_2', '97_3', '97_4', '113_0', '113_1', '113_2', '113_3', '113_4', '129_0', '129_1', '129_2', '129_3', '129_4', '145_0', '145_1', '145_2', '145_3', '145_4', '161_0', '161_1', '161_2', '161_3', '161_4', '177_0', '177_1', '177_2', '177_3', '177_4', '193_0', '193_1', '193_2', '193_3', '193_4', '209_0', '209_1', '209_2', '209_3', '209_4', '225_0', '225_1', '225_2', '225_3', '225_4', '241_0', '241_1', '241_2', '241_3', '241_4', '257_0', '257_1', '257_2', '257_3', '257_4'] 
 ['49_0', '49_1', '49_2', '49_3', '49_4', '49_5', '49_6', '49_7', '49_8', '49_9', '65_0', '65_1', '65_2', '65_3', '65_4', '65_5', '65_6', '65_7', '65_8', '65_9', '81_0', '81_1', '81_2', '81_3', '81_4', '81_5', '81_6', '81_7', '81_8', '81_9', '97_0', '97_1', '97_2', '97_3', '97_4', '97_5', '97_6', '97_7', '97_8', '97_9', '113_0', '113_1', '113_2', '113_3', '113_4', '113_5', '113_6', '11

### avg

In [ ]:
avgs = np.stack( (modis_per_year[2000]['avg'][:14],
           modis_per_year[2001]['avg'][3:17],
           modis_per_year[2002]['avg'][3:17],
           modis_per_year[2003]['avg'][3:17],
           modis_per_year[2004]['avg'][3:17],
           modis_per_year[2005]['avg'][3:17],
           modis_per_year[2006]['avg'][3:17],
           modis_per_year[2007]['avg'][3:17],
           modis_per_year[2008]['avg'][3:17],
           modis_per_year[2009]['avg'][3:17],
           modis_per_year[2010]['avg'][3:17],
           modis_per_year[2011]['avg'][3:17],
           modis_per_year[2012]['avg'][3:17],
           modis_per_year[2013]['avg'][3:17],
           modis_per_year[2014]['avg'][3:17],
           modis_per_year[2015]['avg'][3:17],
           modis_per_year[2016]['avg'][3:17],
           modis_per_year[2017]['avg'][3:17],
           modis_per_year[2018]['avg'][3:17],
           modis_per_year[2019]['avg'][3:17],
           modis_per_year[2020]['avg'][3:17],
           modis_per_year[2021]['avg'][3:17],
           modis_per_year[2022]['avg'][3:]) )
# avgs # 23 years/lists of 14 day observations
avgs.T # 14 days/lists 23 year observations

array([[0.29967428, 0.30937648, 0.33978463, 0.36227483, 0.40658343,
        0.48012774, 0.50275514, 0.5541114 , 0.66283122, 0.71562194,
        0.72222651, 0.6742683 , 0.62136371, 0.51253235],
       [0.27488467, 0.27890844, 0.29511285, 0.3454182 , 0.39236404,
        0.46005837, 0.51935047, 0.55545573, 0.66838808, 0.69467933,
        0.71979871, 0.68699073, 0.64248427, 0.55720816],
       [0.2696209 , 0.25807396, 0.29891741, 0.33935231, 0.40082188,
        0.44254637, 0.47373898, 0.54417801, 0.61597457, 0.67490888,
        0.68852594, 0.68332764, 0.6470703 , 0.56074115],
       [0.2079076 , 0.25998766, 0.31999631, 0.35427373, 0.41827735,
        0.47423595, 0.51033618, 0.57112992, 0.63532836, 0.68830174,
        0.7010065 , 0.66573785, 0.61285542, 0.53916259],
       [0.25787703, 0.2897237 , 0.31462365, 0.36024622, 0.41611357,
        0.46424055, 0.51054716, 0.59977316, 0.64628622, 0.72069527,
        0.7277878 , 0.70750953, 0.67250377, 0.5767206 ],
       [0.25646652, 0.27223825, 0.3

In [ ]:
df_avgs = pd.DataFrame( zip(modis_per_year.keys(), yield_corn, yield_soybean, yield_wheat, *avgs.T),
                       columns = [ 'year', 'yield_corn', 'yield_soybean', 'yield_wheat', *list(range(49, 273, 16)) ])

In [ ]:
df_avgs

,year,yield_corn,yield_soybean,yield_wheat,49,65,81,97,113,129,145,161,177,193,209,225,241,257
0,2000,8.591008,2.561342,2.823927,0.299674,0.309376,0.339785,0.362275,0.406583,0.480128,0.502755,0.554111,0.662831,0.715622,0.722227,0.674268,0.621364,0.512532
1,2001,8.673266,2.663958,2.701927,0.274885,0.278908,0.295113,0.345418,0.392364,0.460058,0.519350,0.555456,0.668388,0.694679,0.719799,0.686991,0.642484,0.557208
2,2002,8.118010,2.556752,2.356827,0.269621,0.258074,0.298917,0.339352,0.400822,0.442546,0.473739,0.544178,0.615975,0.674909,0.688526,0.683328,0.647070,0.560741
3,2003,8.924730,2.276952,2.971268,0.207908,0.259988,0.319996,0.354274,0.418277,0.474236,0.510336,0.571130,0.635328,0.688302,0.701006,0.665738,0.612855,0.539163
4,2004,10.063628,2.840494,2.902680,0.257877,0.289724,0.314624,0.360246,0.416114,0.464241,0.510547,0.599773,0.646286,0.720695,0.727788,0.707510,0.672504,0.576721
5,2005,9.285273,2.896130,2.823190,0.256467,0.272238,0.319851,0.367782,0.421282,0.455442,0.511564,0.595489,0.679299,0.713899,0.705743,0.696155,0.654856,0.536098
6,2006,9.357833,2.881782,2.598712,0.255455,0.290278,0.317984,0.356646,0.418583,0.468456,0.498606,0.565122,0.651256,0.690523,0.682627,0.680580,0.655628,0.569127
7,2007,9.458417,2.806695,2.704637,0.221122,0.275828,0.347883,0.366519,0.411522,0.473419,0.541890,0.592407,0.681693,0.701437,0.698391,0.677150,0.638068,0.531598
8,2008,9.621053,2.671862,3.014640,0.224796,0.256002,0.292174,0.332168,0.383002,0.443516,0.486440,0.540970,0.613237,0.677245,0.711755,0.708011,0.672484,0.603466
9,2009,10.318039,2.959524,2.980516,0.239662,0.281166,0.312572,0.318933,0.378764,0.447520,0.485402,0.568330,0.628433,0.708474,0.731594,0.730101,0.692396,0.597850


### 5 bins

In [ ]:
np.concatenate( modis_per_year[2000]['bins_5'][:14] ) # these are just lined up for each day

array([3.7210e+03, 5.0755e+04, 8.9870e+03, 6.4900e+02, 1.0000e+00,
       2.9360e+03, 4.9920e+04, 1.0363e+04, 1.1210e+03, 1.0000e+00,
       2.0890e+03, 4.5075e+04, 1.4940e+04, 2.3040e+03, 4.0000e+00,
       1.5280e+03, 4.1223e+04, 1.8499e+04, 3.2380e+03, 6.0000e+00,
       4.8500e+02, 3.3565e+04, 2.5101e+04, 5.3020e+03, 4.2000e+01,
       9.4000e+01, 2.0688e+04, 2.9745e+04, 1.3825e+04, 1.4300e+02,
       1.2100e+02, 1.5093e+04, 3.3378e+04, 1.5788e+04, 1.1500e+02,
       1.5000e+01, 6.6820e+03, 3.2406e+04, 2.5079e+04, 3.1300e+02,
       4.0000e+01, 3.6080e+03, 1.1501e+04, 4.4582e+04, 4.7640e+03,
       8.2000e+01, 3.3620e+03, 8.7580e+03, 2.9860e+04, 2.2433e+04,
       1.8900e+02, 3.5890e+03, 8.9470e+03, 2.3499e+04, 2.8271e+04,
       4.0300e+02, 6.0340e+03, 1.2406e+04, 2.6810e+04, 1.8842e+04,
       3.8000e+02, 8.3430e+03, 1.4887e+04, 3.6215e+04, 4.6700e+03,
       6.3800e+02, 1.3993e+04, 3.0788e+04, 1.8661e+04, 4.1500e+02])

In [ ]:
bins_5 = np.stack( (np.concatenate( modis_per_year[2000]['bins_5'][:14] ),
                    np.concatenate( modis_per_year[2001]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2002]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2003]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2004]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2005]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2006]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2007]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2008]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2009]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2010]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2011]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2012]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2013]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2014]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2015]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2016]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2017]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2018]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2019]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2020]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2021]['bins_5'][3:17] ),
                    np.concatenate( modis_per_year[2022]['bins_5'][3:] )) )
# bins_5 # 23 years/lists of 14 day observations # 23 x (14*5=70)
bins_5.T # 14 days/lists 23 year observations # 70 x 23

array([[ 3721.,  6734., 11084., ..., 10308., 15145., 15885.],
       [50755., 26499., 45156., ..., 33512., 37088., 34518.],
       [ 8987.,  4038.,  6888., ...,  6859.,  5411.,  5565.],
       ...,
       [30788., 24999., 28344., ..., 27297., 30701., 21127.],
       [18661., 28843., 27799., ..., 26734., 26090., 32395.],
       [  415.,   192.,   393., ...,   550.,   313.,  1776.]])

In [ ]:
df_bins_5 = pd.DataFrame( zip(modis_per_year.keys(), yield_corn, yield_soybean, yield_wheat, *bins_5.T),
                         columns = [ 'year', 'yield_corn', 'yield_soybean', 'yield_wheat', *get_bin_column_list(5) ])

In [ ]:
df_bins_5

,year,yield_corn,yield_soybean,yield_wheat,49_0,49_1,49_2,49_3,49_4,65_0,...,241_0,241_1,241_2,241_3,241_4,257_0,257_1,257_2,257_3,257_4
0,2000,8.591008,2.561342,2.823927,3721.0,50755.0,8987.0,649.0,1.0,2936.0,...,380.0,8343.0,14887.0,36215.0,4670.0,638.0,13993.0,30788.0,18661.0,415.0
1,2001,8.673266,2.663958,2.701927,6734.0,26499.0,4038.0,505.0,1.0,7338.0,...,483.0,7360.0,13464.0,33863.0,9325.0,600.0,9861.0,24999.0,28843.0,192.0
2,2002,8.118010,2.556752,2.356827,11084.0,45156.0,6888.0,543.0,5.0,14058.0,...,340.0,5794.0,14753.0,35061.0,8547.0,383.0,7576.0,28344.0,27799.0,393.0
3,2003,8.924730,2.276952,2.971268,23726.0,25381.0,6195.0,742.0,4.0,12201.0,...,518.0,8461.0,15339.0,37214.0,2963.0,571.0,10172.0,30737.0,22272.0,743.0
4,2004,10.063628,2.840494,2.902680,10677.0,31355.0,3923.0,694.0,4.0,6266.0,...,408.0,4501.0,13096.0,32124.0,14366.0,261.0,7232.0,25127.0,31447.0,428.0
5,2005,9.285273,2.896130,2.823190,14290.0,36967.0,7268.0,951.0,26.0,11049.0,...,437.0,4904.0,12800.0,40002.0,6352.0,463.0,8290.0,34764.0,20884.0,94.0
6,2006,9.357833,2.881782,2.598712,12382.0,38977.0,6303.0,330.0,0.0,7327.0,...,355.0,6523.0,12313.0,34096.0,11208.0,328.0,8673.0,24508.0,30668.0,318.0
7,2007,9.458417,2.806695,2.704637,16703.0,23991.0,5082.0,211.0,1.0,7773.0,...,445.0,5811.0,14983.0,37890.0,5366.0,678.0,9561.0,33176.0,21035.0,45.0
8,2008,9.621053,2.671862,3.014640,17349.0,23518.0,6655.0,424.0,4.0,13611.0,...,406.0,4997.0,11479.0,33755.0,13858.0,597.0,5954.0,19328.0,37610.0,1006.0
9,2009,10.318039,2.959524,2.980516,12880.0,37660.0,3985.0,584.0,2.0,6228.0,...,401.0,4613.0,10750.0,28890.0,19841.0,414.0,6262.0,21122.0,36048.0,649.0


### 10 bins

In [ ]:
np.concatenate( modis_per_year[2000]['bins_10'][:14] ) # these are just lined up for each day

array([6.3300e+02, 3.0880e+03, 3.5146e+04, 1.5609e+04, 6.3020e+03,
       2.6850e+03, 5.6700e+02, 8.2000e+01, 1.0000e+00, 0.0000e+00,
       4.5700e+02, 2.4790e+03, 3.4495e+04, 1.5425e+04, 7.0530e+03,
       3.3100e+03, 1.0050e+03, 1.1600e+02, 1.0000e+00, 0.0000e+00,
       4.4000e+01, 2.0450e+03, 2.8522e+04, 1.6553e+04, 9.8880e+03,
       5.0520e+03, 2.0090e+03, 2.9500e+02, 4.0000e+00, 0.0000e+00,
       4.1000e+01, 1.4870e+03, 2.4208e+04, 1.7015e+04, 1.1895e+04,
       6.6040e+03, 2.7390e+03, 4.9900e+02, 6.0000e+00, 0.0000e+00,
       0.0000e+00, 4.8500e+02, 1.4595e+04, 1.8970e+04, 1.5242e+04,
       9.8590e+03, 4.1570e+03, 1.1450e+03, 4.2000e+01, 0.0000e+00,
       0.0000e+00, 9.4000e+01, 3.8710e+03, 1.6817e+04, 1.6377e+04,
       1.3368e+04, 1.0783e+04, 3.0420e+03, 1.4300e+02, 0.0000e+00,
       1.0000e+00, 1.2000e+02, 2.3210e+03, 1.2772e+04, 1.7089e+04,
       1.6289e+04, 1.2173e+04, 3.6150e+03, 1.1500e+02, 0.0000e+00,
       0.0000e+00, 1.5000e+01, 1.6060e+03, 5.0760e+03, 1.5412e

In [ ]:
bins_10 = np.stack( (np.concatenate( modis_per_year[2000]['bins_10'][:14] ),
                     np.concatenate( modis_per_year[2001]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2002]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2003]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2004]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2005]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2006]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2007]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2008]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2009]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2010]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2011]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2012]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2013]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2014]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2015]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2016]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2017]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2018]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2019]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2020]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2021]['bins_10'][3:17] ),
                     np.concatenate( modis_per_year[2022]['bins_10'][3:] )) )
# bins_10 # 23 years/lists of 14 day observations # 23 x (14*10=140)
bins_10.T # 14 days/lists 23 year observations # 140 x 23

array([[  633.,  5127.,  6300., ...,  7835., 10264., 12786.],
       [ 3088.,  1607.,  4784., ...,  2473.,  4881.,  3099.],
       [35146., 15211., 31190., ..., 22222., 25776., 23569.],
       ...,
       [ 7522., 11261.,  7276., ..., 10927.,  7325., 16751.],
       [  415.,   192.,   393., ...,   550.,   313.,  1776.],
       [    0.,     0.,     0., ...,     0.,     0.,     0.]])

In [ ]:
df_bins_10 = pd.DataFrame( zip(modis_per_year.keys(), yield_corn, yield_soybean, yield_wheat, *bins_10.T),
                          columns = [ 'year', 'yield_corn', 'yield_soybean', 'yield_wheat', *get_bin_column_list(10) ])

In [ ]:
df_bins_10

,year,yield_corn,yield_soybean,yield_wheat,49_0,49_1,49_2,49_3,49_4,49_5,...,257_0,257_1,257_2,257_3,257_4,257_5,257_6,257_7,257_8,257_9
0,2000,8.591008,2.561342,2.823927,633.0,3088.0,35146.0,15609.0,6302.0,2685.0,...,0.0,638.0,4510.0,9483.0,16467.0,14321.0,11139.0,7522.0,415.0,0.0
1,2001,8.673266,2.663958,2.701927,5127.0,1607.0,15211.0,11288.0,2905.0,1133.0,...,0.0,600.0,2743.0,7118.0,10932.0,14067.0,17582.0,11261.0,192.0,0.0
2,2002,8.118010,2.556752,2.356827,6300.0,4784.0,31190.0,13966.0,5728.0,1160.0,...,0.0,383.0,2226.0,5350.0,10579.0,17765.0,20523.0,7276.0,393.0,0.0
3,2003,8.924730,2.276952,2.971268,19212.0,4514.0,17208.0,8173.0,4919.0,1276.0,...,1.0,570.0,3463.0,6709.0,14392.0,16345.0,13524.0,8748.0,743.0,0.0
4,2004,10.063628,2.840494,2.902680,7745.0,2932.0,18556.0,12799.0,3096.0,827.0,...,2.0,259.0,2082.0,5150.0,9925.0,15202.0,19391.0,12056.0,428.0,0.0
5,2005,9.285273,2.896130,2.823190,10436.0,3854.0,25382.0,11585.0,5783.0,1485.0,...,0.0,463.0,2171.0,6119.0,15178.0,19586.0,15017.0,5867.0,94.0,0.0
6,2006,9.357833,2.881782,2.598712,9309.0,3073.0,25941.0,13036.0,5327.0,976.0,...,0.0,328.0,3167.0,5506.0,9367.0,15141.0,18806.0,11862.0,318.0,0.0
7,2007,9.458417,2.806695,2.704637,13683.0,3020.0,14616.0,9375.0,4150.0,932.0,...,0.0,678.0,2907.0,6654.0,14425.0,18751.0,14900.0,6135.0,45.0,0.0
8,2008,9.621053,2.671862,3.014640,14204.0,3145.0,15752.0,7766.0,5036.0,1619.0,...,0.0,597.0,2142.0,3812.0,6630.0,12698.0,20917.0,16693.0,1006.0,0.0
9,2009,10.318039,2.959524,2.980516,9063.0,3817.0,28531.0,9129.0,3172.0,813.0,...,0.0,414.0,1960.0,4302.0,8288.0,12834.0,18776.0,17272.0,649.0,0.0


### 20 bins

In [ ]:
np.concatenate( modis_per_year[2000]['bins_20'][:14] ) # these are just lined up for each day

array([3.5200e+02, 2.8100e+02, 3.6100e+02, 2.7270e+03, 2.0987e+04,
       1.4159e+04, 9.8930e+03, 5.7160e+03, 3.7850e+03, 2.5170e+03,
       1.7250e+03, 9.6000e+02, 4.0100e+02, 1.6600e+02, 6.3000e+01,
       1.9000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.9300e+02, 2.6400e+02, 3.4500e+02, 2.1340e+03, 2.0296e+04,
       1.4199e+04, 9.4700e+03, 5.9550e+03, 4.2890e+03, 2.7640e+03,
       1.9370e+03, 1.3730e+03, 6.9800e+02, 3.0700e+02, 9.2000e+01,
       2.4000e+01, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       3.4000e+01, 1.0000e+01, 1.1600e+02, 1.9290e+03, 1.5188e+04,
       1.3334e+04, 8.9170e+03, 7.6360e+03, 5.6940e+03, 4.1940e+03,
       3.1050e+03, 1.9470e+03, 1.2990e+03, 7.1000e+02, 2.5700e+02,
       3.8000e+01, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       1.8000e+01, 2.3000e+01, 6.5000e+01, 1.4220e+03, 1.0743e+04,
       1.3465e+04, 9.1600e+03, 7.8550e+03, 6.7320e+03, 5.1630e+03,
       3.9010e+03, 2.7030e+03, 1.7390e+03, 1.0000e+03, 4.2200e

In [ ]:
bins_20 = np.stack( (np.concatenate( modis_per_year[2000]['bins_20'][:14] ),
                     np.concatenate( modis_per_year[2001]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2002]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2003]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2004]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2005]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2006]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2007]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2008]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2009]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2010]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2011]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2012]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2013]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2014]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2015]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2016]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2017]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2018]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2019]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2020]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2021]['bins_20'][3:17] ),
                     np.concatenate( modis_per_year[2022]['bins_20'][3:] )) )
# bins_20 # 23 years/lists of 14 day observations # 23 x (14*20=280)
bins_20.T # 14 days/lists 23 year observations # 280 x 23

array([[3.5200e+02, 4.5660e+03, 4.1070e+03, ..., 6.9020e+03, 8.2120e+03,
        1.0596e+04],
       [2.8100e+02, 5.6100e+02, 2.1930e+03, ..., 9.3300e+02, 2.0520e+03,
        2.1900e+03],
       [3.6100e+02, 4.6800e+02, 1.4600e+03, ..., 6.9200e+02, 1.6900e+03,
        1.4070e+03],
       ...,
       [0.0000e+00, 0.0000e+00, 2.0000e+00, ..., 7.0000e+00, 1.0000e+00,
        3.8000e+01],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [ ]:
df_bins_20 = pd.DataFrame( zip(modis_per_year.keys(), yield_corn, yield_soybean, yield_wheat, *bins_20.T),
                          columns = [ 'year', 'yield_corn', 'yield_soybean', 'yield_wheat', *get_bin_column_list(20) ])

In [ ]:
df_bins_20

,year,yield_corn,yield_soybean,yield_wheat,49_0,49_1,49_2,49_3,49_4,49_5,...,257_10,257_11,257_12,257_13,257_14,257_15,257_16,257_17,257_18,257_19
0,2000,8.591008,2.561342,2.823927,352.0,281.0,361.0,2727.0,20987.0,14159.0,...,7704.0,6617.0,5704.0,5435.0,4811.0,2711.0,415.0,0.0,0.0,0.0
1,2001,8.673266,2.663958,2.701927,4566.0,561.0,468.0,1139.0,5940.0,9271.0,...,6200.0,7867.0,8991.0,8591.0,7957.0,3304.0,192.0,0.0,0.0,0.0
2,2002,8.118010,2.556752,2.356827,4107.0,2193.0,1460.0,3324.0,15811.0,15379.0,...,7906.0,9859.0,10885.0,9638.0,5183.0,2093.0,391.0,2.0,0.0,0.0
3,2003,8.924730,2.276952,2.971268,16692.0,2520.0,1564.0,2950.0,9966.0,7242.0,...,8663.0,7682.0,6989.0,6535.0,5255.0,3493.0,734.0,9.0,0.0,0.0
4,2004,10.063628,2.840494,2.902680,6290.0,1455.0,915.0,2017.0,8070.0,10486.0,...,7146.0,8056.0,9131.0,10260.0,8431.0,3625.0,428.0,0.0,0.0,0.0
5,2005,9.285273,2.896130,2.823190,9130.0,1306.0,1085.0,2769.0,12657.0,12725.0,...,10006.0,9580.0,8533.0,6484.0,4342.0,1525.0,94.0,0.0,0.0,0.0
6,2006,9.357833,2.881782,2.598712,8021.0,1288.0,754.0,2319.0,13405.0,12536.0,...,6609.0,8532.0,9215.0,9591.0,7854.0,4008.0,318.0,0.0,0.0,0.0
7,2007,9.458417,2.806695,2.704637,11893.0,1790.0,1197.0,1823.0,6903.0,7713.0,...,9468.0,9283.0,8228.0,6672.0,4545.0,1590.0,45.0,0.0,0.0,0.0
8,2008,9.621053,2.671862,3.014640,12706.0,1498.0,1055.0,2090.0,9058.0,6694.0,...,5015.0,7683.0,9943.0,10974.0,10455.0,6238.0,986.0,20.0,0.0,0.0
9,2009,10.318039,2.959524,2.980516,7778.0,1285.0,1003.0,2814.0,16885.0,11646.0,...,5878.0,6956.0,8620.0,10156.0,10769.0,6503.0,647.0,2.0,0.0,0.0


## export to csv

In [ ]:
df_avgs.to_csv(path + 'modis_avgs.csv', index=False)
df_bins_5.to_csv(path + 'modis_bins_5.csv', index=False)
df_bins_10.to_csv(path + 'modis_bins_10.csv', index=False)
df_bins_20.to_csv(path + 'modis_bins_20.csv', index=False)

In [ ]:
df = pd.read_csv(path + 'modis_avgs.csv')

In [ ]:
df

,year,yield_corn,yield_soybean,yield_wheat,49,65,81,97,113,129,145,161,177,193,209,225,241,257
0,2000,8.591008,2.561342,2.823927,0.299674,0.309376,0.339785,0.362275,0.406583,0.480128,0.502755,0.554111,0.662831,0.715622,0.722227,0.674268,0.621364,0.512532
1,2001,8.673266,2.663958,2.701927,0.274885,0.278908,0.295113,0.345418,0.392364,0.460058,0.519350,0.555456,0.668388,0.694679,0.719799,0.686991,0.642484,0.557208
2,2002,8.118010,2.556752,2.356827,0.269621,0.258074,0.298917,0.339352,0.400822,0.442546,0.473739,0.544178,0.615975,0.674909,0.688526,0.683328,0.647070,0.560741
3,2003,8.924730,2.276952,2.971268,0.207908,0.259988,0.319996,0.354274,0.418277,0.474236,0.510336,0.571130,0.635328,0.688302,0.701006,0.665738,0.612855,0.539163
4,2004,10.063628,2.840494,2.902680,0.257877,0.289724,0.314624,0.360246,0.416114,0.464241,0.510547,0.599773,0.646286,0.720695,0.727788,0.707510,0.672504,0.576721
5,2005,9.285273,2.896130,2.823190,0.256467,0.272238,0.319851,0.367782,0.421282,0.455442,0.511564,0.595489,0.679299,0.713899,0.705743,0.696155,0.654856,0.536098
6,2006,9.357833,2.881782,2.598712,0.255455,0.290278,0.317984,0.356646,0.418583,0.468456,0.498606,0.565122,0.651256,0.690523,0.682627,0.680580,0.655628,0.569127
7,2007,9.458417,2.806695,2.704637,0.221122,0.275828,0.347883,0.366519,0.411522,0.473419,0.541890,0.592407,0.681693,0.701437,0.698391,0.677150,0.638068,0.531598
8,2008,9.621053,2.671862,3.014640,0.224796,0.256002,0.292174,0.332168,0.383002,0.443516,0.486440,0.540970,0.613237,0.677245,0.711755,0.708011,0.672484,0.603466
9,2009,10.318039,2.959524,2.980516,0.239662,0.281166,0.312572,0.318933,0.378764,0.447520,0.485402,0.568330,0.628433,0.708474,0.731594,0.730101,0.692396,0.597850
